In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [2]:
response = requests.get('https://data.cityofnewyork.us/resource/hvrh-b6nb.json')

if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:
        print("Not JSON.")
else:
    print("Retrieve failed.")

In [3]:
df = pd.DataFrame.from_records(data)

In [4]:
df.to_csv("green_2016.csv", index=False)

In [5]:
connection_string = "DefaultEndpointsProtocol=https;AccountName=triathlonnyc;AccountKey=wfxBgbmGNXsXavVUF48Zn7sBj5ZNgT60MTArVyDKwn0CdqVebvaYyMWDzKfddso2yX0iiyr2fcK4+AStTsLANQ==;EndpointSuffix=core.windows.net"

In [6]:
container_name = "green"

In [7]:
blob_name = "green_2016"

In [8]:
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [9]:
container_client = blob_service_client.get_container_client(container_name)

In [11]:
# with open("green_2016.csv", "rb") as data:
#     container_client.upload_blob(name = blob_name, data=data)

In [21]:
# The base URL for the NYC Open Data portal's SODA API
base_url = "https://data.cityofnewyork.us/resource/"

# The discovery API endpoint for datasets
discovery_endpoint = "https://api.us.socrata.com/api/catalog/v1"

# Parameters for querying the dataset
params = {
    'domains': 'data.cityofnewyork.us',
    'search_context': 'data.cityofnewyork.us',
    'q': 'yellow taxi 2019'
}

# Send a GET request to the discovery endpoint with the parameters
response = requests.get(discovery_endpoint, params=params)

# Check if the request was successful
if response.status_code == 200:
    datasets = response.json()
    
    # Extract dataset identifiers and construct API endpoints
    for dataset in datasets['results']:
        resource = dataset['resource']
        dataset_id = resource['id']
        dataset_name = resource['name']
        api_endpoint = f"{base_url}{dataset_id}.json"
        
        print(f"Dataset Name: {dataset_name}")
        print(f"API Endpoint: {api_endpoint}")
else:
    print(f"Failed to retrieve datasets, status code: {response.status_code}")

Dataset Name: 2019 Yellow Taxi Trip Data
API Endpoint: https://data.cityofnewyork.us/resource/2upf-qytp.json
Dataset Name: Medallion Taxi Initial Inspection Schedule
API Endpoint: https://data.cityofnewyork.us/resource/sp7n-275u.json


In [31]:
def DatasetRetrieval2(year, colour):
    base_url = "https://data.cityofnewyork.us/resource/"

    # The discovery API endpoint for datasets
    discovery_endpoint = "https://api.us.socrata.com/api/catalog/v1"

    # Parameters for querying the dataset
    params = {
        'domains': 'data.cityofnewyork.us',
        'search_context': 'data.cityofnewyork.us',
        'q': f'{colour} taxi {year}'
    }

    # Send a GET request to the discovery endpoint with the parameters
    response = requests.get(discovery_endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        datasets = response.json()
        
        # Extract dataset identifiers and construct API endpoints
        for dataset in datasets['results']:
            resource = dataset['resource']
            dataset_id = resource['id']
            dataset_name = resource['name']
            api_endpoint = f"{base_url}{dataset_id}.csv"
            
            if set(year).issubset(set(dataset_name)) and set(colour.lower()).issubset(set(dataset_name.lower())):
                print(dataset_name)
            return api_endpoint
    else:
        print(f"Failed to retrieve datasets, status code: {response.status_code}")

In [32]:
DatasetRetrieval2('2016', 'green')

2016 Green Taxi Trip Data


'https://data.cityofnewyork.us/resource/hvrh-b6nb.csv'

In [33]:
a = DatasetRetrieval2('2016', 'green')

print(a)

2016 Green Taxi Trip Data
https://data.cityofnewyork.us/resource/hvrh-b6nb.csv
